In [2]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
data = pd.DataFrame(pd.read_excel(r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\Testdatav2.xlsx"))

intersection_columns = ['InterSection1', 'InterSection2', 'InterSection3', 'InterSection4']
traffic_columns = ['TrafficDensity1','TrafficDensity2','TrafficDensity3','TrafficDensity4']

x = data[['InterSection1']]
y = data['TrafficDensity1']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
#print(y_pred)

data_test1 = data['InterSection1'].values.reshape(-1, 1)
y_pred = dt.predict(data_test1)
# print(y_pred)

data_test2 = data['InterSection2'].values.reshape(-1, 1)
y_pred = dt.predict(data_test2)
# print(y_pred)

data_test3 = data['InterSection3'].values.reshape(-1, 1)
y_pred = dt.predict(data_test3)
# print(y_pred)

data_test4 = data['InterSection4'].values.reshape(-1, 1)
y_pred = dt.predict(data_test4)
# print(y_pred)

import pandas as pd

# Assuming y_pred for each data_test is available as mentioned in your code
data_test1 = data['InterSection1'].values.reshape(-1, 1)
y_pred1 = dt.predict(data_test1)

data_test2 = data['InterSection2'].values.reshape(-1, 1)
y_pred2 = dt.predict(data_test2)

data_test3 = data['InterSection3'].values.reshape(-1, 1)
y_pred3 = dt.predict(data_test3)

data_test4 = data['InterSection4'].values.reshape(-1, 1)
y_pred4 = dt.predict(data_test4)

# Create a DataFrame with the predicted values
df = pd.DataFrame({
    'InterSection1': y_pred1,
    'InterSection2': y_pred2,
    'InterSection3': y_pred3,
    'InterSection4': y_pred4
})

#print(df)

label_mapping = {
    'Very Light': 1,
    'Light': 2,
    'Moderate': 3,
    'Heavy': 4,
    'Very Heavy': 5
}

# Iterate over rows and assign ranks
for i, row in df.iterrows():
    ranks = [label_mapping[label] for label in row.values]
    df.loc[i] = ranks

#print(df)

# Assuming y_pred for each data_test is available as mentioned in your code
data_test1 = data['InterSection1'].values.reshape(-1, 1)
y_pred1 = dt.predict(data_test1)

data_test2 = data['InterSection2'].values.reshape(-1, 1)
y_pred2 = dt.predict(data_test2)

data_test3 = data['InterSection3'].values.reshape(-1, 1)
y_pred3 = dt.predict(data_test3)

data_test4 = data['InterSection4'].values.reshape(-1, 1)
y_pred4 = dt.predict(data_test4)

# Create a DataFrame with the predicted values
df = pd.DataFrame({
    'Lane1': y_pred1,
    'Lane2': y_pred2,
    'Lane3': y_pred3,
    'Lane4': y_pred4
})

# Iterate over rows, assign ranks, sort ranks, and assign intersection names
for i, row in df.iterrows():
    ranks = [label_mapping[label] for label in row.values]
    sorted_ranks = sorted(range(len(ranks)), key=lambda x: (ranks[x], x), reverse=True)
    intersections = ['Lane' + str(index + 1) for index in sorted_ranks]
    df.loc[i, 'Sorted_Result'] = ', '.join(intersections)
#........................................................................
df = pd.DataFrame(data)
# Extract intersection data into arrays
intersection_arrays = []
for _, row in df.iterrows():
    intersection_data = []
    for i in range(1, 5):  # Assuming there are always 4 intersections
        intersection_data.append(row[f'InterSection{i}'])
    intersection_arrays.append(intersection_data)

#print(intersection_arrays)

#print(df)
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Green light times data
green_light_times = {
    (0,): 0,
    (1,): 10,
    (2,): 15,
    (3,): 20,
    (4,): 25,
    (5,): 30,
    (6,): 35,
    (7,): 40,
    (8,): 45
}

# Convert green light times data to arrays for training
x_train = np.array(list(green_light_times.keys())).reshape((-1, 1))  # Reshape for compatibility with sklearn
y_train = np.array(list(green_light_times.values()))

# Instantiate PolynomialFeatures object to generate polynomial features
poly = PolynomialFeatures(degree=2)

# Transform the input features to include polynomial features
x_train_poly = poly.fit_transform(x_train)

# Train polynomial regression model
model = LinearRegression()
model.fit(x_train_poly, y_train)

# Define a function to predict green light time for a given number of cars
def predict_green_light_time(cars):
    total_time = 0
    for car in cars:
        car_array = np.array([[car]])  # Reshape the input for compatibility with model prediction
        car_array_poly = poly.transform(car_array)  # Transform input for polynomial regression
        predicted_time = model.predict(car_array_poly)[0]
        total_time += predicted_time
    return total_time

# Test input
test_input = intersection_arrays
results_df = pd.DataFrame(columns=['Sorted_Result', 'Predicted_Times', 'Total_Predicted_Time'])
##################################
results_list = []

for i, row in enumerate(test_input):
    predicted_times = [model.predict(poly.transform(np.array([[car]])))[0] for car in row]
    sorted_predicted_times = sorted(predicted_times, reverse=True)  # Sort predicted times in descending order
    total_time = sum(sorted_predicted_times)
    total_time_rounded = round(total_time)
    
    # Create a dictionary to store the results for each row
    result_dict = {}
    
    # Split sorted result into four lanes
    sorted_result = [f'Lane{index + 1}' for index in np.argsort(predicted_times)[::-1]]
    for j in range(4):
        result_dict[f'Sorted_Lane_{j + 1}'] = sorted_result[j]
    
    # Add columns for predicted times for each lane
    for j, time in enumerate(sorted_predicted_times):
        result_dict[f'Predicted_Time_{j + 1}'] = round(time)
    
    # Add column for total time
    result_dict['Total_Predicted_Time'] = total_time_rounded
    
    results_list.append(result_dict)

# Convert list of dictionaries to DataFrame
results_df = pd.DataFrame(results_list)

# Reorder columns so that total time appears at the last column
column_order = [col for col in results_df.columns if col != 'Total_Predicted_Time'] + ['Total_Predicted_Time']
results_df = results_df[column_order]

# Print or do further processing with results_df
#print(results_df)
# Save the DataFrame to an Excel file
results_df.to_excel(r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx", index=False)


c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  

In [36]:
results_df

,Sorted_Lane_1,Sorted_Lane_2,Sorted_Lane_3,Sorted_Lane_4,Predicted_Time_1,Predicted_Time_2,Predicted_Time_3,Predicted_Time_4,Total_Predicted_Time
0,Lane1,Lane4,Lane2,Lane3,44,8,8,2,62
1,Lane1,Lane4,Lane2,Lane3,40,25,14,8,88
2,Lane1,Lane4,Lane2,Lane3,40,31,20,14,105
3,Lane4,Lane2,Lane1,Lane3,40,40,40,20,140
4,Lane2,Lane4,Lane1,Lane3,44,31,31,25,131
5,Lane3,Lane4,Lane2,Lane1,31,25,25,25,107
6,Lane4,Lane1,Lane3,Lane2,40,40,36,25,141
7,Lane1,Lane3,Lane4,Lane2,44,40,31,25,141
8,Lane3,Lane2,Lane4,Lane1,44,31,8,8,91
9,Lane4,Lane1,Lane2,Lane3,40,25,8,2,75


In [37]:
# Save the DataFrame to an Excel file
results_df.to_excel(r"C:\Users\User\Desktop\AI_traffic_dataset\predicted_traffic_times.xlsx", index=False)